### Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import imageio

# If do not install ffmpeg in your local mahine
# You are getting ssl verification error please go to ffmpeg site and download manuel
imageio.plugins.ffmpeg.download()
 
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
 
sys.path.append("..")
from object_detection.utils import ops as utils_ops
import datetime

### Tensorflow Paths

Please add your personal tensorflow object detection api path.

In [3]:
RESEARCH_PATH = '/Users/fatihcelik/tensorflow-object-detection/TensorFlow/models/research/'
MODELS_PATH = '/Users/fatihcelik/tensorflow-object-detection/TensorFlow/models/research/object_detection/'

### Object Detection Imports

In [4]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## MODEL PREPARATIONS

### Model Variables

In [5]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2018_01_28'

MODEL_FILE = MODEL_NAME + '.tar.gz'

DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 40

### Download or Upload Model

In [6]:
# If not download model please download tensorflow model file
# opener = urllib.request.URLopener()
# opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

# If model file downloaded please extract tar.gz
tar_file = tarfile.open(MODEL_FILE)

for file in tar_file.getmembers():
    
    file_name = os.path.basename(file.name)
    
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())

### Load a (frozen) Tensorflow model into memory

In [7]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

### Loading label map

In [8]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

### Helper

In [9]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getData()).reshape((im_height, im_width, 3)).astype(np.uint8)

### Detection

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        input_video = 'test1'
        video_reader = imageio.get_reader('input/%s.mp4'%input_video)
        video_writer = imageio.get_writer('output/%s_tracking.mp4'%input_video, fps=10)
        t0 = datetime.datetime.now()
        n_frames = 0
        for frame in video_reader:
            image_np = frame
            n_frames+=1
            
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes, scores, classes, num) = sess.run([detection_boxes, detection_classes, detection_classes, num_detections], feed_dict={image_tensor: image_np_expanded})
            vis_util.visualize_boxes_and_labels_on_image_array(image_np, np.squeeze(boxes), np.squeeze(classes).astype(np.int32), np.squeeze(scores), category_index, use_normalized_coordinates=True, line_thickness=8)
            video_writer.append_data(image_np)
            
        fps = n_frames/(datetime.datetime.now()-t0).total_seconds()
        print('Frame Processed: %s, Speed: %s fps'%(n_frames, fps))
            
        video_writer.close()